In [1]:
import os
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import pillow_heif
import re
from collections import defaultdict


In [2]:
# Registrar suporte HEIF/HEIC no Pillow
pillow_heif.register_heif_opener()

file = 'ImagensVisaoComp'

# Verificar se o diretório existe
if not os.path.exists(file):
    print(f"O diretório '{file}' não existe.")
    exit()

imagens = os.listdir(file)

# Filtrar apenas os arquivos de imagem
image_files = [img for img in imagens if img.lower().endswith(('.jpg', '.jpeg', '.png', '.heic'))]


In [3]:
def split_name(image_name):
    name_parts = image_name.split('-')
    if len(name_parts) < 4:
        print(f"Nome da imagem '{image_name}' não está no formato esperado.")
        return None
    object_name = name_parts[0].strip()
    day_or_night = name_parts[1].strip()
    environment = name_parts[2].strip()
    background = name_parts[3].strip().split('.')[0]
    return object_name, day_or_night, environment, background

def create_image_table(image_files, folder_path): 
    data = {
        'Nome do objeto': [],
        'Dia/Noite': [],
        'Ambiente': [],
        'Fundo': [],
        'Tamanho (KB)': [],
        'Dimensão (px)': [],
    }
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        parsed_data = split_name(image_file)
        if parsed_data is None:
            print(f"Nome da imagem '{image_file}' não está no formato esperado.")
            continue
        
        object_name, day_or_night, environment, background = parsed_data

        try:
            image = Image.open(image_path)
            size_kb = os.path.getsize(image_path) / 1024
            width, height = image.size
            dimension = f"{width}x{height}"

            data['Nome do objeto'].append(object_name)
            data['Dia/Noite'].append(day_or_night)
            data['Ambiente'].append(environment)
            data['Fundo'].append(background)
            data['Tamanho (KB)'].append(int(size_kb)) 
            data['Dimensão (px)'].append(dimension)
            
        except Exception as e:
            print(f"Erro ao processar a imagem '{image_file}': {e}")
    
    df = pd.DataFrame(data)
    df.to_csv('tabela_imagens.csv', index=False)
    
    return df

def tabela_global_sumaria(image_files, folder_path):
    # Dicionário para armazenar os dados globais
    data = {
        'Nomes dos objetos': [],
        'Número de classes': 0,
        'Número de Imagens': 0,
        'Tamanho da base (bytes)': 0,
        'Resolução das imagens': None,  # Resolução da primeira imagem ou uma forma de agregação
    }
    
    selected_object = set()  # Usar set para manter apenas nomes únicos
    total_size_kb = 0  # Acumulador de tamanho
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        parsed_data = split_name(image_file)
        if parsed_data is None:
            print(f"Nome da imagem '{image_file}' não está no formato esperado.")
            continue
        
        object_name, day_or_night, environment, background = parsed_data
        
        match = re.match(r"([a-zA-Z]+)\d+", object_name)  # Captura apenas as letras
        if match:
            name = match.group(1)
            selected_object.add(name)  # Adicionar o nome ao conjunto
            
        try:
            image = Image.open(image_path)
            size_kb = os.path.getsize(image_path) / 1024
            total_size_kb += size_kb  # Acumular o tamanho total

            # Capturar a resolução da primeira imagem processada
            if data['Resolução das imagens'] is None:
                width, height = image.size
                dimension = f"{width}x{height}"
                data['Resolução das imagens'] = dimension  # Salvar a resolução

        except Exception as e:
            print(f"Erro ao processar a imagem '{image_file}': {e}")
            continue

    # Após o loop, consolidar os dados em uma única linha
    data['Nomes dos objetos'] = list(selected_object)  # Lista de objetos únicos
    data['Número de classes'] = len(selected_object)  # Quantidade de classes
    data['Número de Imagens'] = len(image_files)  # Quantidade total de imagens
    data['Tamanho da base (bytes)'] = int(total_size_kb)  # Tamanho total

    # Criar um DataFrame com uma única linha
    df = pd.DataFrame([data])
    df.to_csv('tabela_global_sumaria.csv', index=False)

    return df


def tabela_detalhada_classe(image_files):
    # Dicionário para armazenar os dados detalhados
    data = defaultdict(lambda: {
        'Qntd de Objetos': 0,
        'Variações de fundo': set(),
        'variações de iluminação': set(),
        'Total de amostras': 0,
    })

    selected_name = []

    for image_file in image_files:
        
        parsed_data = split_name(image_file)
        if parsed_data is None:
            print(f"Nome da imagem '{image_file}' não está no formato esperado.")
            continue
        
        object_name, day_or_night, environment, background = parsed_data
        
        data[object_name]['Qntd de Objetos'] += 1  
        data[object_name]['Variações de fundo'].add(background)  
        data[object_name]['variações de iluminação'].add(day_or_night)  
        data[object_name]['Total de amostras'] += 1  

    # Criar lista para DataFrame
    table_data = []
    for object_name, stats in data.items():
        table_data.append({
            'Nome do objeto': object_name,
            'Qntd de Objetos': stats['Qntd de Objetos'],
            'Variações de fundo': ', '.join(stats['Variações de fundo']),
            'variações de iluminação': ', '.join(stats['variações de iluminação']),
            'Total de amostras': stats['Total de amostras'],
        })
    
    # Criar um DataFrame com os dados detalhados
    df = pd.DataFrame(table_data)
    df.to_csv('tabela_detalhada_classe.csv', index=False)
    
    return df


In [ ]:
num_columns = 5
num_rows = (len(image_files) + num_columns - 1) // num_columns
fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))

for idx, image_file in enumerate(image_files):
    image_path = os.path.join(file, image_file)
    image = Image.open(image_path)

    row = idx // num_columns
    col = idx % num_columns

    parsed_data = split_name(image_file)
    if parsed_data is None:
        nome_processado = image_file
    else:
        object_name, day_or_night, environment, background = parsed_data
        nome_processado = f"{object_name} {day_or_night} {environment} {background}"

    axs[row, col].imshow(image)
    axs[row, col].axis('off')
    axs[row, col].set_title(nome_processado, fontsize=18)

plt.tight_layout()
plt.show()

In [5]:
# df = create_image_table(image_files, file)
# print(df)

# print("")

print('Tabela Global Sumária')
df = tabela_global_sumaria(image_files, file)
print(df)

print("")

print('Tabela Detalhada por classe')
df = tabela_detalhada_classe(image_files)
print(df)

Tabela Global Sumária
                  Nomes dos objetos  Número de classes  Número de Imagens  \
0  [Relogio, Tenis, caneca, oculos]                  4                144   

   Tamanho da base (bytes) Resolução das imagens  
0                   332683             4080x3072  

Tabela Detalhada por classe
   Nome do objeto  Qntd de Objetos      Variações de fundo  \
0         caneca1               12  fundo3, fundo1, fundo2   
1         caneca2               12  fundo3, fundo1, fundo2   
2         caneca3               12  fundo3, fundo1, fundo2   
3         oculos1               12  fundo3, fundo1, fundo2   
4         oculos2               12  fundo3, fundo1, fundo2   
5         oculos3               12  fundo3, fundo1, fundo2   
6        Relogio1               12  fundo3, fundo1, fundo2   
7        Relogio2               12  fundo3, fundo1, fundo2   
8        Relogio3               12  fundo3, fundo1, fundo2   
9          Tenis1               12  fundo3, fundo1, fundo2   
10        